# Notebook 10 - Apprentissage par renforcement / Taxi autonome

CSI4106 Intelligence Artificielle   
Automne 2020  
Preparé par Julian Templeton, Caroline Barrière et Joel Muteba

***INTRODUCTION***:  
Dans ce notebook, nous explorerons l'utilisation de l'apprentissage par renforcement pour aider un agent à résoudre une tâche spécifique dans un environnement fourni par la [bibliothèque OpenAI's Gym](https://gym.openai.com/). Cette bibliothèque fournit un certain nombre d'environnements, nous pouvons entraîner notre modèle IA à maîtriser un ou plusieurs de ces environnements. Dans ce notebook, nous explorerons un scénario dans lequel un taxi situé dans une grille doit être contrôlé par un agent pour prendre un passager situé dans l'une de quatre positions et déposer le passager dans l'une de trois autres positions.

Pour vous familiariser avec le problème de la cabine autonome abordé dans ce notebook, veuillez vous rendre sur le site https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/ et lire la section 1 (récompenses), section 2 (espace d'états) qui vous fera comprendre pourquoi il y a 500 états possibles, section 3 (espace d'action) qui décrit les actions possibles.

Tout au long du notebook, nous travaillerons avec une approche de base et une approche basée sur le Q-Learning. Cela fournira un aperçu de la façon dont Q-Learning peut être appliqué aux problèmes et comment un agent peut utiliser l'apprentissage par renforcement pour résoudre des problèmes dans un environnement.

**Lors de la soumission de ce notebook, assurez-vous de NE PAS réinitialiser les sorties de l'exécution du code (et n'oubliez pas d'enregistrer le notebook avec ctrl + s).**

**Afin de faciliter l'installation, vous exécuterez à nouveau ce notebook dans Google Colab, PAS sur votre ordinateur local.**

***Devoir***:  
Parcourez le notebook en exécutant chaque cellule, une à la fois.
Recherchez **(TO DO)** pour les tâches que vous devez effectuer. Ne modifiez pas le code en dehors des questions auxquelles vous êtes invité à répondre à moins que cela ne vous soit spécifiquement demandé. Une fois que vous avez terminé, signez le notebook (à la fin du notebook) et soumettez-le.

*Le notebook sera noté sur 30.
Chaque **(TO DO)** a un certain nombre de points qui lui sont associés.*
***

**1.0 - Mise en place du jeu de taxi**   

Pour commencer le notebook, nous devrons configurer et explorer l'environnement avec lequel notre agent travaillera. OpenAI's Gym propose de nombreuses expériences différentes à utiliser. Celles-ci vont des actes d'équilibrage aux voitures autonomes en passant par le jeu simple d'Atari. Malheureusement, toutes les options disponibles ne peuvent pas être facilement utilisées. Beaucoup peuvent prendre des heures d'entraînement pour commencer à voir des résultats passionnants. Chacune de ces expériences utilise des agents qui peuvent être entraînés par apprentissage par renforcement pour maîtriser comment exécuter la tâche spécifiée. Les méthodes utilisées peuvent aller de la simple utilisation du Q-Learning à l'utilisation plus complexe d'un ou plusieurs modèles de Deep Learning qui fonctionnent en conjonction avec des techniques d'apprentissage par renforcement.

Une expérience simple, mais intéressante, implique un taxi contrôlé par l'IA qui doit prendre et déposer un passager. C'est le problème que nous allons explorer tout au long du notebook. Le code utilisé dans tout le notebook provient de [cet exemple](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/) et a été modifié conséquemment.

Pour commencer, nous installerons certains des packages dont nous aurons besoin pour exécuter le programme.

In [4]:
# Install the necessary libraries
!pip install cmake "gym[atari]" scipy

In [195]:
# Import the necessary libraries
import random
from random import choice
import gym
import numpy as np
from IPython.display import clear_output

Avec toutes les bibliothèques installées, nous allons maintenant utiliser le programme Taxi fourni par Gym. Ci-dessous, nous allons importer Gym, charger le programme comme environnement actif et montrer une image représentant l'état actuel du programme.

D'après l'image ci-dessous, il existe quatre emplacements clés différents dans l'environnement, représentés par *R*, *G*, *B* et *Y*. La lettre qui est en gras en bleu représente l'endroit où le passager actuel doit être récupéré et la lettre en gras en violet représente l'endroit où le passager souhaite déposer. Le bloc jaune représente la cellule dans laquelle se trouve actuellement le taxi. Par conséquent, le taxi doit d'abord récupérer le passager et le déposer au lieu de dépose. Lorsqu'un passager est dans le taxi, il devient vert jusqu'à ce que le passager soit déposé.

In [6]:
# Load the environment
env = gym.make("Taxi-v3").env
# Render the current state of the program
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Ensuite, nous allons réinitialiser l'état de l'environnement et montrer à nouveau l'état actuel. Nous imprimons également le nombre total d'actions disponibles pour notre agent (défini comme *l'espace d'action*) et *l'espace d'état* qui représente l'état du programme (où se trouve la cabine, le passager, le lieu de prise en charge et le lieu où déposer).

In [7]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


Intuitivement, nous voulons que notre agent apprenne quelle action entreprendre dans un état spécifique. Plus précisément, quelles mesures devraient être prises en fonction de l'emplacement du taxi par rapport à l'emplacement du passager et au lieu où déposer. Les six actions possibles que le taxi peut effectuer à un étape donnée sont:

Action = 0: Aller vers le Sud

Action = 1: Aller vers le Nord

Action = 2: Aller vers l'Est

Action = 3: Aller vers l'Ouest

Action = 4: Ramasser/Prendre le passager

Action = 5: Déposer le passager

Vous trouverez ci-dessous un exemple de définition de l'état avec un codage spécifique et le rendu de cet état.

In [8]:
# The encoding below represents: (taxi row, taxi column, passenger index, destination index)
state = env.encode(3, 1, 2, 0) 
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



L'exemple suivant montre comment définir un état avec le passager dans le taxi.

In [10]:
#### The encoding below represents: (taxi row, taxi column, passenger index, destination index)
state = env.encode(0, 1, 4, 0) 
print("State:", state)

env.s = state
env.render()

State: 36
+---------+
|R:_| : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



#### **(TO DO) Q1**
Maintenant que nous avons vu comment définir un état via un encodage, vous devrez définir l'état pour qu'il corresponde aux descriptions ci-dessous et les montrer.

a) Réglez le passager à la position G, le passager souhaitant être déposé à la position R et le taxi positionné à un point aléatoire sur la grille (la position sélectionnée du taxi doit être choisie au hasard). Après avoir défini la position, montrez le rendu (avec render) de l'état.

b) Réglez le passager pour qu'il soit dans le taxi (à n'importe quelle position sans lettre dessus) et réglez le point de débarquement des passagers sur la position B. Après avoir défini la position, restituez (avec render) l'état.

**(TO DO) Q1 (a) - 2 points**    
a) Réglez le passager à la position G, le passager souhaitant être déposé à la position R et le taxi positionné à un point aléatoire sur la grille (la position sélectionnée du taxi doit être choisie au hasard). Après avoir défini la position, montrez le rendu (avec render) de l'état.

In [197]:
# TODO (remember to use random coordinates within the grid for the taxi)...
randRow = random.randint(0, 4)
randCol = random.randint(0, 4)
state = env.encode(randRow, randCol, 1, 0)
print("State:", state)

env.s = state
env.render()

State: 64
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



**(TO DO) Q1 (b) - 2 points**

b) Réglez le passager pour qu'il soit dans le taxi (à n'importe quelle position sans lettre dessus) et réglez le point de débarquement des passagers sur la position B. Après avoir défini la position, restituez (avec render) l'état.

In [198]:
# TODO ...
randRow = random.randint(0, 4)
randCol = random.randint(0, 4)
if randRow ==0 :
  randCol = choice([i for i in range(0,4) if i not in [0,4]])
  state = env.encode(randRow, randCol, 4,3)
elif randRow ==4:
  randCol = choice([i for i in range(0,4) if i not in [0,3]])
  state = env.encode(randRow, randCol, 4,3)
else :
  state = env.encode(randRow, randCol, 4,3)

print("State:", state)

env.s = state
env.render()

State: 219
+---------+
|R: | : :G|
| : | : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+



Pour chaque action que le taxi peut entreprendre, nous avons une liste représentant les informations clés par rapport à ce qui se passera lorsqu'une action est effectuée. Après avoir effectué une action, l'agent recevra une récompense ou une pénalité. Cette récompense ou pénalité indiquera à l'agent à quel point sa décision d'effectuer l'action spécifiée était bonne ou mauvaise.

Ci-dessous, nous affichons un dictionnaire qui contient toutes les actions possibles ainsi que les informations suivantes dans les tuples correspondants:

(

   La probabilité de prendre cette action,
  
   L'état résultant après avoir effectué cette action,
  
   La récompense pour avoir pris cette action,
  
   Si le programme se terminera ou non lors de l'exécution de l'action

)

Exemple de tuple: (1.0, 328, -1, False)

In [36]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

Bien que non affiché par le code ci-dessus, si le taxi contient le passager et est au-dessus du point de débarquement, la récompense pour l'action de déposer le passager est de 20.

**2.0 - Approche de base du jeu du taxi**   

Pour commencer, nous allons effectuer la simulation du scénario de taxi avec une approche de base qui n'utilise pas Q-Learning. Cette approche fonctionnera simplement en sélectionnant une action aléatoire disponible à chaque pas de temps, quel que soit l'état actuel. Nous préparerons également une méthode de lecture de toutes les images d'un épisode pour voir comment l'agent contrôle le taxi dans le scénario.

In [37]:
def run_single_simulation_baseline(env, state, disable_prints=False):
    '''
    Given the environment and a specific state, randomly select an action for the taxi
    to perform until the goal is completed.
    '''
    if not disable_prints:
        print("Testing for simulation: {}".format(state))
    # Set the state of the environment
    env.s = state
    # Used to hold all information for a single time step (including the image data)
    frames = []
    # Used to determine when the simulation has been completed
    done = False
    # Determines the number of times steps that the application has been run for
    time_steps = 0
    # The total values used to determine how many times the agent mistakenly
    # picks up no one or attempts to dropoff no passenger or attempts to
    # dropoff a passenger in the wrong position.
    penalties, reward = 0, 0
    # Run until the passenger has been picked up and dropped off in the target location
    while not done:
        # Perform a random action from the set of available actions in the environment
        action = env.action_space.sample()
        # From performing the action, retrieve the new state, the reward from taking the action,
        # whether the simulation is complete, and other information from performing the action.
        state, reward, done, info = env.step(action)
        # If an incorrect dropoff or pickup is performed, increment the penalty count
        if reward == -10:
            penalties += 1
        # Put each rendered frame into dict to use for animating the process and
        # tracking the details over the run
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        # Increment the time step count
        time_steps += 1
    # State the total number of steps taken and the total penalties that have occured.
    if not disable_prints:
        print("Timesteps taken: {}".format(time_steps))
        print("Penalties incurred: {}".format(penalties))
    # Return the frame data, the total penalties, and the total time steps
    return frames, penalties, time_steps

Une fois l'approche de base définie, nous allons exécuter un test avec cette approche pour voir combien de temps il faut à un agent utilisant cette approche pour trouver une solution pour la simulation 328 et combien de pénalités majeures l'agent reçoit.

In [201]:
state = 328
# Run a test and collect all frames from the run
frames, _, _ = run_single_simulation_baseline(env, state)

Testing for simulation: 328
Timesteps taken: 173
Penalties incurred: 69


Après avoir effectué une simulation et récupéré les résultats, nous pouvons utiliser les images obtenues à partir de la simulation et les transmettre à la fonction *print_frames* ci-dessous pour afficher une animation contenant toutes les images ainsi que les informations qui provenaient de chaque étape et temps auquel l'image correspond .

Pour le premier épisode que vous visualisez, il est recommandé d'exécuter l'ensemble du processus à une vitesse plus lente (telle que 0,3 ou 0,5 dans l'appel de veille). Cependant, vous êtes libre d'augmenter la vitesse du processus en réduisant le nombre dans l'appel de la fonction de veille dans la fonction *print_frames* ci-dessous.

In [39]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    '''
    For each frame, show the frame and display the timestep it occurred at,
    the number of the active state, the action selected, adn the corresponding reward.
    '''
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        # Can adjust speed here
        sleep(.3)
# Print the frames from the episode
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1958
State: 0
Action: 5
Reward: 20


**(TO DO) Q2**

a) En utilisant l'état défini à partir de Q1 (a), récupérer les frames/images correspondantes obtenues en utilisant l'approche de base ci-dessus. Ensuite, affichez ces cadres(frames ou images).

b) En utilisant l'état défini à partir de Q1 (b), récupérer les frames/images correspondantes obtenues en utilisant l'approche de base ci-dessus. Ensuite, affichez ces cadres.

**(TO DO) Q2 (a) - 2 points**   
a) En utilisant l'état défini à partir de Q1 (a), récupérer les frames/images correspondantes obtenues en utilisant l'approche de base ci-dessus. Ensuite, affichez ces cadres(frames ou images).

In [202]:
# TODO: Retrieve the corresponding frames from running the simulation starting from the state found in Q1 (a). Then show those frames.
state = 64
frames, _, _ = run_single_simulation_baseline(env, state)

Testing for simulation: 64
Timesteps taken: 999
Penalties incurred: 307


**(TO DO) Q2 (b) - 2 points**

a) En utilisant l'état défini à partir de Q1 (b), récupérer les frames/images correspondantes obtenues en utilisant l'approche de base ci-dessus. Ensuite, affichez ces cadres(frames ou images).

In [203]:
# TODO: Retrieve the corresponding frames from running the simulation starting from the state found in Q1 (b). Then show those frames.
state = 219
frames, _, _ = run_single_simulation_baseline(env, state)

Testing for simulation: 219
Timesteps taken: 471
Penalties incurred: 149


Avec la possibilité de simuler des exécutions uniques d'un épisode avec l'approche de base, nous allons maintenant définir une fonction que nous utiliserons pour évaluer les performances générales du modèle de base lorsqu'il est exécuté sur plusieurs épisodes. La fonction *evaluate_agent_baseline* ci-dessous accepte comme entrée le nombre total d'épisodes sélectionnés au hasard à exécuter avec l'environnement, exécute les épisodes aléatoires, affiche le nombre moyen de pas de temps pris par épisode avec les pénalités moyennes encourues et renvoie les données d'image.

***Si la fonction evaluate_agent_baseline semble s'exécuter trop longtemps (plusieurs minutes, pas une seule), arrêtez l'exécution en cliquant sur le bouton en haut à gauche de la cellule de code en cours d'exécution et réexécutez-la.***

In [190]:
def evaluate_agent_baseline(episodes, env):
    '''
    Given a number of episodes and an environment, run the specified
    number of episodes, where each run begins with a random state, display the
    naverage timesteps per episode and the average penalties per episode, and output
    the frames to be displayed.
    '''
    total_time_steps, total_penalties = 0, 0
    frames = []
    # Run through the total number of episodes
    for _ in range(episodes):
        # Get a random state
        state = env.reset()
        # Run the simulation, obtaining the results
        frame_data, penalties, time_steps = run_single_simulation_baseline(env, state, True)
        # Update the tracked data over all simulations
        total_penalties += penalties
        total_time_steps += time_steps
        frames = frames + frame_data
    print(f"Results after {episodes} episodes:")
    print(f"Average timesteps per episode: {total_time_steps / episodes}")
    print(f"Average penalties per episode: {total_penalties / episodes}")
    return frames

**(TO DO) Q3**

a) Utilisez la fonction *evaluate_agent_baseline* définie ci-dessus pour exécuter 100 épisodes aléatoires pour l'environnement.

b) D'après les résultats de la Q3 (a), comment l'approche de référence a-t-elle fonctionné et pourquoi pensez-vous qu'elle a bien ou mal fonctionné? Expliquez les temps moyens par épisode et les pénalités moyennes par épisode.

c) Sans passer à une approche d'apprentissage par renforcement, comment l'approche de base peut-elle être modifiée pour être légèrement meilleure?

**(TO DO) Q3 (a) - 1 point**

a) Utilisez la fonction *evaluate_agent_baseline* définie ci-dessus pour exécuter 100 épisodes aléatoires pour l'environnement.

In [191]:
# TODO ...
frames = evaluate_agent_baseline(100,env)

Results after 100 episodes:
Average timesteps per episode: 2512.88
Average penalties per episode: 807.92


**(TO DO) Q3 (b) - 1 point**

b) D'après les résultats de la Q3 (a), comment l'approche de référence a-t-elle fonctionné et pourquoi pensez-vous qu'elle a bien ou mal fonctionné? Expliquez les temps moyens par épisode et les pénalités moyennes par épisode.

TODO ...     
Par reference l'action et choisi de facon aleatoire en fonction de l'etat actuer et les possibiliter d'effectuer une action qui a deja ete prise et a conduit a une penalite, cela ce produit tant que la destination final n'est pas atteinte. Le temps moyen par episode correspond au nombre de deplacement moyen par le taxi pour recuper et deposer le passager a la destination.

**(TO DO) Q3 (c) - 1 point**

c) Sans passer à une approche d'apprentissage par renforcement, comment l'approche de base peut-elle être modifiée pour être légèrement meilleure?

TODO ...  L'approche pourrait etre legerement meilleur en sauvegardant les deplacements que le taxi effectue, pour faire un deplacement seulement s'il na pas deja ete effectuer

**3.0 - Entraîner un agent avec Q-Learning pour jouer au jeu du taxi**   

Maintenant que nous avons eu un agent utilisant l'approche de base pour compléter la simulation de taxi, nous demanderons à l'agent d'utiliser Q-Learning pour essayer d'appliquer une approche d'apprentissage par renforcement au problème. Pour démarrer le processus, nous allons créer une matrice de valeurs Q pour chaque possibilité d'état d'action (en l'initialisant à zéro). L'agent mettra à jour cette matrice lors de l'entraînement et aura besoin de la réinitialiser chaque fois qu'il souhaite réinitialiser son entraînement.

In [192]:
# Initialize the table of Q values for the state-action pairs
q_table = np.zeros([env.observation_space.n, env.action_space.n])

Une fois la matrice des valeurs Q initialisée, nous allons maintenant définir la fonction d'apprentissage qui ajuste les valeurs Q dans *q_table*. Le processus d'apprentissage consiste à exécuter un certain nombre de simulations aléatoires et à mettre à jour les valeurs Q pour chaque état via Q-Learning.

Il existe un certain nombre d'hyperparamètres utilisés par la fonction d'apprentissage:

- *alpha*: paramètre d'apprentissage (vous devrez le décrire dans une question ultérieure).
- *gamma*: le paramètre de remise de récompense à long terme.
- *epsilon*: paramètre d'Exploitation/Exploration (vous devrez le décrire dans une question ultérieure).
- *num_simulations*: Représente le nombre d'épisodes aléatoires à générer pour que les agents les utilisent pour mettre à jour ses valeurs Q.

Ainsi, en exécutant cet algorithme, un agent peut apprendre les valeurs Q à utiliser lorsqu'il travaille avec d'autres épisodes.

In [193]:
def train_agent(alpha, gamma, epsilon, num_simulations):
    '''
    Trains an agent by updating its Q values for a total of num_simulations
    episodes with the alpha, gamma, and epsilon hyperparameters. 
    '''
    # For plotting metrics
    all_time_steps = []
    all_penalties = []
    # Generate the specified number of episodes
    for i in range(1, num_simulations + 1):
        # Generate a new state by resetting it
        state = env.reset()
        # Variables tracked (time steps, total penalties, the reward value)
        time_steps, penalties, reward, = 0, 0, 0
        done = False
        # Run the simulation 
        while not done:
            # Select a random action is the randomly selected number from a
            # uniform distribution is less than epsilon
            if random.uniform(0, 1) < epsilon:
                action = env.action_space.sample() # Explore action space
            # Otherwise use the currently learned Q values
            else:
                action = np.argmax(q_table[state]) # Exploit learned values
            # Retrieve the relevant information after performing the action
            next_state, reward, done, info = env.step(action) 
            # Retrieve the old Q value and the maximum Q value from the next state
            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])
            # Update the current Q value
            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value
            # Track anytime an incorrect dropoff or pickup is made
            if reward == -10:
                penalties += 1
            # Proceed to the next state and time step
            state = next_state
            time_steps += 1
        # Display progress for each 100 episodes
        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {i}")
    print("Training finished.\n")

Nous utilisons maintenant la fonction d'entraînement avec un ensemble d'hyperparamètres pour entraîner l'agent avec Q-Learning afin d'améliorer potentiellement les performances au fil du temps.

In [204]:
# Hyperparameters
alpha = 0.1
gamma = 0.5
epsilon = 0.1
num_simulations = 100000
# Train the agent
train_agent(alpha, gamma, epsilon, num_simulations)

Episode: 100000
Training finished.



Après l'entraînement, nous pouvons regarder les valeurs Q qui ont été obtenues dans notre table state-action pour un état spécifique. Ci-dessous, nous voyons que chaque valeur Q pour les six actions possibles disponibles pour l'état 328 a été mise à jour en conséquence.

**(TO DO) Q4 - 2 points**

Ci-dessous, nous imprimons les valeurs Q disponibles pour les six actions à l'état 328 et nous rendons(venant de render)/montrons cet état pour le visualiser. Sur la base des valeurs Q disponibles (en supposant que nous sommes en mode d'exploitation), quelle action serait la prochaine à être sélectionnée (ou s'il y a des égalités, listez toutes les actions possibles qui seraient envisagées)? Certaines des actions contenant des valeurs Q plus élevées semblent-elles problématiques si jamais elles sont sélectionnées? Pourquoi ou pourquoi pas?

In [205]:
print(q_table[328])
env.s = 328
env.render()

[ -1.98513267  -1.95703125  -1.98408534  -1.97698365 -10.25056685
 -10.58847597]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


TODO ...    
La prochaine valeur selectionner vers le nord n'est pas problematique car les valeurs sont choisi du minimum vers le plus grand.

Une fois l'entraînement terminé, nous pouvons maintenant évaluer l'approche Q-Learning dans une méthode similaire que nous avons utilisée pour évaluer l'approche de base. En passant le nombre d'épisodes à tester et l'environnement, nous générons ce nombre d'épisodes aléatoires et faisons la moyenne des résultats obtenus en exécutant l'approche Q-Learning pour terminer les épisodes. Contrairement à l'entraînement, il est important de noter que les hyperparamètres qui y sont utilisés ne sont pas utilisés ici. L'agent utilise simplement la valeur Q maximale à chaque étape pour déterminer quelle action entreprendre à un pas de temps donné.

***Si la fonction evaluate_agent_QL semble s'exécuter trop longtemps (une minute ou plus), arrêtez l'exécution en cliquant sur le bouton en haut à gauche de la cellule de code en cours d'exécution et réexécutez-la. Cela se produit parce que l'entraînement était suffisant pour définir des valeurs Q valides et a entraîné une impasse pour un état spécifique.***

In [206]:
def evaluate_agent_QL(episodes, env):
    '''
    Given a number to specify how many random states to run and the environment to use,
    display the averaged metrics obtained from the tests and return the frames obtained from the tests.
    '''
    total_time_steps, total_penalties = 0, 0
    frames = []
    for _ in range(episodes):
        # Generate a random state to use
        state = env.reset()
        # The information collected throughout the run
        time_steps, penalties, reward = 0, 0, 0
        # Determines when the episode is complete
        done = False
        # Run through the episode until complete
        while not done:
            # Select the action containing the maximum Q value
            action = np.argmax(q_table[state])
            # Run that action and retrieve the reward and other details
            state, reward, done, info = env.step(action)

            # Put each rendered frame into dict for animation
            frames.append({
                'frame': env.render(mode='ansi'),
                'state': state,
                'action': action,
                'reward': reward
                }
            )
            # Specify whether the agent incorrectly chose to pick up or dropoff a passenger
            if reward == -10:
                penalties += 1
            # Increment the current time step
            time_steps += 1
        # Track the totals
        total_penalties += penalties
        total_time_steps += time_steps
    # Display the performance over the tests
    print(f"Results after {episodes} episodes:")
    print(f"Average timesteps per episode: {total_time_steps / episodes}")
    print(f"Average penalties per episode: {total_penalties / episodes}")
    # Return the frames to allow a user to view the runs
    return frames

**(TO DO) Q5**

a) Exécutez *evaluate_agent_QL* pendant 100 épisodes pour récupérer le nombre moyen de pas de temps et la pénalité moyenne après l'entraînement.

b) Compte tenu de vos résultats de Q5 (a), comment les résultats observés des tests se comparent-ils aux tests de l'approche de base de Q3 (a)? Plus précisément, quel agent est le plus performant par rapport au nombre moyen de pénalités tout au long des tests et quel agent est capable de résoudre les problèmes plus rapidement (en moyenne).

**(TO DO) Q5 (a) - 1 point**

a) Exécutez *evaluate_agent_QL* pendant 100 épisodes pour récupérer le nombre moyen de pas de temps et la pénalité moyenne après l'entraînement.

In [207]:
# TODO ...
frames = evaluate_agent_QL(100,env)

Results after 100 episodes:
Average timesteps per episode: 12.93
Average penalties per episode: 0.0


**(TO DO) Q5 (b) - 2 points**

b) Compte tenu de vos résultats de Q5 (a), comment les résultats observés des tests se comparent-ils aux tests de l'approche de base de Q3 (a)? Plus précisément, quel agent est le plus performant par rapport au nombre moyen de pénalités tout au long des tests et quel agent est capable de résoudre les problèmes plus rapidement (en moyenne).

TODO ... 

On peut observer avec les rusultat que QL est plus performant que baseline car on obtient moins de penalites pour chaques episode et cela prend moins de temps par episoder pour prendre et deposer le passager a la destination.

**4.0 - Test de différents hyperparamètres**   

Nous allons maintenant essayer de re-entraîner l'agent en utilisant différentes configurations avec les hyperparamètres. Cela vous permettra d'explorer leur impact sur le Q-Learning et de comprendre leur objectif pendant l'entraînement.  

**(TO DO) Q6**     
Ci-dessous, nous explorons différentes variations de quatre hyperparamètres utilisés par l'approche Q-Learning pour mieux comprendre leur impact sur l'entraînement. Lorsque vous répondez aux questions, ***veillez à régler correctement les hyperparamètres***.

a) Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et l'entraînement pour seulement **35000 épisodes** (avec les mêmes valeurs alpha, gamma et epsilon utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un autre test pour 100 épisodes dans l'environnement.

b) Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et en s'entraînant pour **100000 épisodes**, mais avec une **valeur epsilon de 0,8** (avec les mêmes valeurs alpha et gamma utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un autre test pour 100 épisodes dans l'environnement.

c) Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et en s'entraînant pour **100000 épisodes**, mais avec une **valeur alpha de 0,7** (avec les mêmes valeurs gamma et epsilon utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un autre test pour 100 épisodes dans l'environnement.

d) Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et en entraînant pour **100000 épisodes**, mais avec une **valeur gamma de 0,15** (avec les mêmes valeurs alpha et epsilon utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un autre test pour 100 épisodes dans l'environnement.

e) En vous basant sur vos connaissances, décrivez quelles sont les rôles des valeurs alpha et epsilon dans la fonction d'entraînement (c'est-à-dire qu'est-ce qu'elles font/leurs impacts).

f) En utilisant les résultats obtenus à partir de vos tests en Q6 (a), (b), (c) et (d), ainsi que les résultats initiaux de Q5, expliquez les impacts de la modification du nombre d'épisodes entraînés (moins d'épisodes vs plus d'épisodes), la valeur alpha (petite vs grande), la valeur gamma (petite vs grande) et la valeur epsilon (petite vs grande). Même si les différences dans les comparaisons sont mineures, indiquez-les.

A noter, ci-dessous sont les valeurs d'hyperparamètres initiales utilisées à la section 3.0 de ce notebook à utiliser comme référence:

*alpha* = 0,1

*gamma* = 0,5

*epsilon* = 0,1

*num_simulations* = 100000

**(TO DO) Q6 (a) - 2 points**

a) Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et l'entraînement pour seulement **35000 épisodes** (avec les mêmes valeurs alpha, gamma et epsilon utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un autre test pour 100 épisodes dans l'environnement.

In [208]:
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.1
gamma = 0.5
epsilon = 0.1
num_simulations = 35000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 100 episodes
frames=evaluate_agent_QL(100,env)

Episode: 35000
Training finished.

Results after 100 episodes:
Average timesteps per episode: 13.51
Average penalties per episode: 0.0


**(TO DO) Q6 (b) - 2 points**

b) Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et en s'entraînant pour **100000 épisodes**, mais avec une **valeur epsilon de 0,8** (avec les mêmes valeurs alpha et gamma utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un autre test pour 100 épisodes dans l'environnement.

In [211]:
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.1
gamma = 0.5
epsilon = 0.8
num_simulations = 100000 
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 100 episodes
frames=evaluate_agent_QL(100,env)

Episode: 100000
Training finished.

Results after 100 episodes:
Average timesteps per episode: 13.32
Average penalties per episode: 0.0


**(TO DO) Q6 (c) - 2 points**

c) Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et en s'entraînant pour **100000 épisodes**, mais avec une **valeur alpha de 0,7** (avec les mêmes valeurs gamma et epsilon utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un autre test pour 100 épisodes dans l'environnement.

In [212]:
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.7
gamma = 0.5
epsilon = 0.1
num_simulations = 100000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 100 episodes
frames=evaluate_agent_QL(100,env)

Episode: 100000
Training finished.

Results after 100 episodes:
Average timesteps per episode: 12.82
Average penalties per episode: 0.0


**(TO DO) Q6 (d) - 2 points**

d) Re-entraînez l'agent en réinitialisant les valeurs d'apprentissage Q et en entraînant pour **100000 épisodes**, mais avec une **valeur gamma de 0,15** (avec les mêmes valeurs alpha et epsilon utilisées dans la section 3.0 de ce notebook). Ensuite, effectuez un autre test pour 100 épisodes dans l'environnement.

In [213]:
# TODO: Reset q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
# TODO: Retrain with the specified hyperparameters
# Hyperparameters
alpha = 0.1
gamma = 0.15
epsilon = 0.1
num_simulations = 100000
train_agent(alpha, gamma, epsilon, num_simulations)
# TODO: Test for 100 episodes
frames=evaluate_agent_QL(100,env)

Episode: 100000
Training finished.

Results after 100 episodes:
Average timesteps per episode: 12.42
Average penalties per episode: 0.0


**(TO DO) Q6 (e) - 2 points**

e) En vous basant sur vos connaissances, décrivez quelles sont les rôles des valeurs alpha et epsilon dans la fonction d'entraînement (c'est-à-dire qu'est-ce qu'elles font/leurs impacts).

TODO ...  

Nous remarquons que le taux au quel l'agent executes les nouvelles action est representer par epsillon. Alors que alpha represente entre la valeur de Q courante et la valeur de Q de l'etat suivant.

**(TO DO) Q6 (f) - 4 points**

f) En utilisant les résultats obtenus à partir de vos tests en Q6 (a), (b), (c) et (d), ainsi que les résultats initiaux de Q5, expliquez les impacts de la modification du nombre d'épisodes entraînés (moins d'épisodes vs plus d'épisodes), la valeur alpha (petite vs grande), la valeur gamma (petite vs grande) et la valeur epsilon (petite vs grande). Même si les différences dans les comparaisons sont mineures, indiquez-les.

TODO ...   Nous remarquons que les difference dans le temps par episode varrient tres legerement quand le nombre d'episode et le meme entre Q5 et Q6 avec un legere avantage de l'approche Q5 sauf quand on on reduit les valeur de alpha ou gamma a ce moment la le temps par episode et plus interessant dans Q6.

***SIGNATURE:***
Mon nom est Valentin Magot.
Mon numéro d'étudiant est 8843488
Je confirme être l'auteur de ce travail.